# Creation / Alteration de table

In [ ]:
CREATE TABLE IT_Equipment_2 
as
select * from `IT_Equipment`

In [ ]:
CREATE TABLE it_equipment_match_equipment
as
SELECT it.asset_id asset_id_it,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.modele,
       it.reperage_id,
       it.hauteur_u,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, aa.Rack Rack_aa, aa.Slot_Begin Slot_Begin_aa, aa.Slot_End Slot_End_aa

       
FROM   `IT_Equipment` it 
INNER JOIN `assets_new`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
WHERE (aa.API = 'FRVLB0C2' or aa.API = 'FRTCRAVA')
ORDER BY it.status ASC

# VIEW

it_equipment_all_retired_asset : jointure entre it_equipment et tous les assets retired avec ou sans status.

In [ ]:
CREATE VIEW it_equipment_all_retired_asset
as
SELECT it.asset_id asset_id_it,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.modele,
       it.reperage_id,
       it.hauteur_u,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, aa.Rack Rack_aa, aa.Slot_Begin Slot_Begin_aa, aa.Slot_End Slot_End_aa

       
FROM   `IT_Equipment` it 
INNER JOIN `assets_new`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
OR
    aa.nom_court = it.nom 
OR
    aa.nom_long = it.nom
OR
    aa.nom_court = it.short_name
OR
    aa.nom_long = it.short_name
    
WHERE aa.etat like 'retired'
ORDER BY it.status ASC

Version condensée de IT_Equipment + jointure avec asset_new uniquement sur la position

In [ ]:
CREATE VIEW IT_Equipment_match_position_reduced
as
SELECT it.reperage_id, count(it.reperage_id) as RowNumber

FROM   `IT_Equipment` it 
INNER JOIN `assets_new`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
WHERE (aa.API = 'FRVLB0C2' or aa.API = 'FRTCRAVA')
GROUP BY it.reperage_id

Regroupement de tous les matchs de position et/ou nom entre IT_equipment sans filtre avec rajout du nombre de match par equipment : Exemple, nom = CH002 correspond à une seule ligne sur IT_Equipment mais à 32 lignes sur Asset = 32 lignes générés avec toutes les infos essentielles

In [ ]:
CREATE VIEW IT_Equipment_match_position
AS
SELECT it.asset_id asset_id_it,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.modele,
       it.reperage_id,
       it.hauteur_u,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, aa.etat, it2.RowNumber

       
FROM   `IT_Equipment` it 
INNER JOIN `assets_new`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced` it2
ON it.reperage_id = it2.reperage_id

WHERE (aa.API = 'FRVLB0C2' or aa.API = 'FRTCRAVA') and it2.rowNumber > 1
ORDER BY it2.rowNumber desc, Reperage_ID, it.status ASC

Match sur position et nom 

In [ ]:
CREATE VIEW `IT_Equipment_match_no_proliant_c7000`
SELECT it.asset_id asset_id_it,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.modele,
       it.reperage_id,
       it.hauteur_u,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, aa.Rack Rack_aa, aa.Slot_Begin Slot_Begin_aa, aa.Slot_End Slot_End_aa, aa.etat

       
FROM   `IT_Equipment` it 
INNER JOIN `assets_new`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
OR
    aa.nom_court = it.nom
    
    
WHERE it.status is null
ORDER BY it.status ASC

# Query

### "EXADATA : No match"

Si "Fabricant" = "Oracle" -> status "EXADATA : No match"

In [ ]:
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
where it.fabricant like '%oracle%' and (asset_id is null or asset_id ='0') -- and status is null

### "Netapp : No match"

Si "Fabricant" = "Netapp" -> status "Netapp : No match"

In [ ]:
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
where it.fabricant like '%netapp%' and (asset_id is null or asset_id ='0') -- status is null

### "PCP : No Match"

Si "Nom" contient "_Rack" -> "PCP : No Match"

In [ ]:
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
where (it.Nom like '%_Rack%' or it.short_name like '%_Rack%') and (asset_id is null or asset_id ='0') and status is null

# Update

### "EXADATA : No match"

Si "Fabricant" = "Oracle" -> status "EXADATA : No match"

In [ ]:
57 rows
UPDATE `IT_Equipment` it
SET it.status = 'EXADATA : No match'
where it.fabricant like '%oracle%' and (asset_id is null or asset_id ='0') and status is null

### "Netapp : No match"

Si "Fabricant" = "Netapp" -> status "Netapp : No match"

In [ ]:
57 rows
UPDATE `IT_Equipment` it
SET it.status = 'Netapp : No match'
where it.fabricant like '%netapp%' and (asset_id is null or asset_id ='0') and status is null

### "PCP : No Match"

Si "Nom" contient "_Rack" -> "PCP : No Match"

In [ ]:
61 rows
UPDATE `IT_Equipment` it
SET it.status = 'PCP : No Match'
where (it.Nom like '%_Rack%' or it.short_name like '%_Rack%') and (asset_id is null or asset_id ='0') and status is null

## Sur Asset.Filtree, prendre la liste de "Fabricant" = "HDS", prendre les 5 lettres près "-" ou "_", pur croiser avec "IT_Equipement" = "Hitachi" en cherchant avec les 5 lettres 

In [ ]:
116 rows
UPDATE `IT_Equipment` it 
INNER JOIN `IT_Equipment_cleaning_Hitachi` hit on hit.it_nom = it.nom
LEFT JOIN `assets_filtree` aa on UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-','')) = hit.clean_name

SET it.status = (
				CASE WHEN aa.asset_id is null and it.status is null then 'Hitashi : No match'
    			WHEN aa.asset_id is not null and it.status is null then concat('HDS : ',aa.asset_id)
    			END 
    			)
                
WHERE it.Fabricant like '%Hitachi%'

# Nombre de match par position 

On effectue une jointure entre une nouvelle table IT_Equipment_2 et asset_new et on check tous les matchs par rapport à la position uniquement

- 1/ View intemédiaire : IT_Equipment_match_position_reduced qui permet de regroupé le nombre de match de position (Salle/Rack/Slot U) par dalle
- 2/ Jointure entre la nouvelle table IT_Equipment_2 et asset_new pour avoir tous les matchs par positions sans distinction particulière sur son état
- 3 / On rajoute avec une seconde jointure le nombre de match par position dans le nouvel attribut status 

####  /!\ La vue IT_Equipment_match_position permet de voir tous les match effectuées avec les correspondances it_equipments et asset /!\

In [ ]:
UPDATE `IT_Equipment_2` it 
INNER JOIN `assets_new`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced` it2
ON it.reperage_id = it2.reperage_id

SET it.status = Concat('Status = MATCH-POSITION : ', it2.rowNumber)


## Match sur les equipements sans status avec un etat dans la CMDB différent de Install

In [ ]:
UPDATE `IT_Equipment` it 
INNER JOIN `assets_new`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
OR
    aa.nom_court = it.nom
    
SET it.status = 'MATCH-CMDB : HORS Status Install'
WHERE it.status is null and (aa.api = 'FRVLB0C2' or aa.api = 'FRTCRAVA')